In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image
from dataset import get_loaders
from PIL import Image
import segmentation_models_pytorch as smp

# ------------------------------
# 1. 損失與評估函數
# ------------------------------
def dice_loss(pred, target, eps=1e-6):
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    return 1 - (2. * intersection + eps) / (union + eps)

def mixed_loss(pred, target):
    bce = nn.BCEWithLogitsLoss()(pred, target)
    return bce + dice_loss(pred, target)

def compute_iou(preds, masks, threshold=0.5):
    preds = (torch.sigmoid(preds) > threshold).float()
    intersection = (preds * masks).sum(dim=(1, 2, 3))
    union = ((preds + masks) > 0).float().sum(dim=(1, 2, 3))
    return (intersection / (union + 1e-6)).mean().item()

def evaluate_prediction_error(preds, masks, px_per_cm=72):
    preds = (torch.sigmoid(preds) > 0.5).float()
    errors = []
    for i in range(preds.size(0)):
        pred_coords = torch.nonzero(preds[i, 0], as_tuple=False)
        true_coords = torch.nonzero(masks[i, 0], as_tuple=False)
        if len(pred_coords) == 0 or len(true_coords) == 0:
            continue
        pred_y = pred_coords[:, 0].float().mean()
        true_y = true_coords[:, 0].float().mean()
        errors.append(torch.abs(pred_y - true_y).item())

    if not errors:
        return 0, 0, 0
    mean_err = np.mean(errors)
    acc_0_5cm = np.mean(np.array(errors) <= (0.5 * px_per_cm)) * 100
    acc_1_0cm = np.mean(np.array(errors) <= (1.0 * px_per_cm)) * 100
    return mean_err, acc_0_5cm, acc_1_0cm

# ------------------------------
# 2. 訓練與預測流程（使用預訓練 Unet）
# ------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
os.makedirs("checkpoints/unetPlusPlus", exist_ok=True)

folds = ['Fold1', 'Fold2', 'Fold3', 'Fold4', 'Fold5']
for fold in folds:
    print(f"=== 開始訓練 {fold} ===")
    model = smp.UnetPlusPlus(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=1
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    train_loader, val_loader = get_loaders('processed_data', fold, batch_size=4)

    best_val_iou = 0
    patience = 30
    early_stop_counter = 0

    for epoch in range(100):
        model.train()
        total_loss = 0
        for imgs, masks in train_loader:
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            loss = mixed_loss(outputs, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # 驗證
        model.eval()
        val_iou = val_err = acc_0_5cm = acc_1_0cm = 0
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs, masks = imgs.to(device), masks.to(device)
                outputs = model(imgs)
                val_iou += compute_iou(outputs, masks)
                mean_err, a05, a10 = evaluate_prediction_error(outputs, masks)
                val_err += mean_err
                acc_0_5cm += a05
                acc_1_0cm += a10

        val_iou /= len(val_loader)
        val_err /= len(val_loader)
        acc_0_5cm /= len(val_loader)
        acc_1_0cm /= len(val_loader)

        print(f"[{fold} - Epoch {epoch+1}] Loss: {total_loss:.2f} | IOU: {val_iou:.4f} | 平均誤差: {val_err:.2f}px | "
              f"誤差<0.5cm: {acc_0_5cm:.2f}% | 誤差<1cm: {acc_1_0cm:.2f}%")

        if val_iou > best_val_iou:
            best_val_iou = val_iou
            early_stop_counter = 0  # reset patience
            torch.save(model.state_dict(), f"checkpoints/unetplusplus/unetplusplus_{fold}_best.pth")
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"🛑 Early stopping at epoch {epoch+1} for {fold}")
                break

    # 儲存整個驗證集所有預測圖像
    out_dir = f"predictions/unetplusplus/{fold}"
    os.makedirs(out_dir, exist_ok=True)
    model.eval()
    count = 1

    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs = imgs.to(device)
            masks = masks.to(device)
            preds = model(imgs)
            preds = (torch.sigmoid(preds) > 0.5).float()

            for i in range(imgs.size(0)):
                orig = to_pil_image(imgs[i][0].cpu())
                gt = to_pil_image(masks[i][0].cpu())
                pred = to_pil_image(preds[i][0].cpu())

                w, h = orig.width, orig.height
                combined = Image.new('L', (w * 3, h))
                combined.paste(orig, (0, 0))
                combined.paste(gt, (w, 0))
                combined.paste(pred, (2 * w, 0))
                combined.save(f"{out_dir}/pred_{count}.png")
                count += 1

print("✅ 使用預訓練 U-Net 完成訓練！")


=== 開始訓練 Fold1 ===
[Fold1 - Epoch 1] Loss: 109.88 | IOU: 0.0593 | 平均誤差: 25.40px | 誤差<0.5cm: 88.89% | 誤差<1cm: 93.06%
[Fold1 - Epoch 2] Loss: 90.69 | IOU: 0.1579 | 平均誤差: 6.20px | 誤差<0.5cm: 100.00% | 誤差<1cm: 100.00%
[Fold1 - Epoch 3] Loss: 84.87 | IOU: 0.2309 | 平均誤差: 6.41px | 誤差<0.5cm: 100.00% | 誤差<1cm: 100.00%
[Fold1 - Epoch 4] Loss: 80.67 | IOU: 0.1906 | 平均誤差: 5.87px | 誤差<0.5cm: 100.00% | 誤差<1cm: 100.00%
[Fold1 - Epoch 5] Loss: 76.65 | IOU: 0.2599 | 平均誤差: 4.90px | 誤差<0.5cm: 100.00% | 誤差<1cm: 100.00%
[Fold1 - Epoch 6] Loss: 72.98 | IOU: 0.2464 | 平均誤差: 5.77px | 誤差<0.5cm: 100.00% | 誤差<1cm: 100.00%
[Fold1 - Epoch 7] Loss: 68.78 | IOU: 0.2817 | 平均誤差: 5.80px | 誤差<0.5cm: 100.00% | 誤差<1cm: 100.00%
[Fold1 - Epoch 8] Loss: 63.60 | IOU: 0.2775 | 平均誤差: 7.38px | 誤差<0.5cm: 100.00% | 誤差<1cm: 100.00%
[Fold1 - Epoch 9] Loss: 58.50 | IOU: 0.3198 | 平均誤差: 6.14px | 誤差<0.5cm: 100.00% | 誤差<1cm: 100.00%
[Fold1 - Epoch 10] Loss: 52.35 | IOU: 0.3697 | 平均誤差: 5.63px | 誤差<0.5cm: 100.00% | 誤差<1cm: 100.00%
[Fold1 - E